処理の流れ

生文 → 形態素解析 → 数値化（ベクトル化）

独立請求項の生文が入力されている列の1行目に「独立請求項」という名前がつけられているexcelデータを 

'sample_data.xlsx' として 'data/'フォルダに保存しておけばサンプルプログラムが動作します。

In [1]:
import os
from google.colab import drive

# googleドライブのマウント 
drive.mount('/content/drive/')

# 作業ファイルをマイドライブに変更
os.chdir('/content/drive/My Drive/') 

Mounted at /content/drive/


In [ ]:
# pandasモジュールを用いたサンプルデータの読み込み

import pandas as pd

data = pd.read_excel('data/sample_data.xlsx')
data.head()

,公報番号(全文リンク),発明等の名称,出願人・権利者(最新),ＩＰＣ(最新),ＦＩ(最新),Ｆターム(最新),独立請求項,筆頭FI,出願日,インキ,機構,受容シート,用途
0,特開2019-218636,染色物の製造方法、染色調整インクジェットインク、及び染色調整インクジェットインクセット,株式会社ミマキエンジニアリング,D06P5/00;C09D11/54;C09D11/32;C09D11/40;D06P5/3...,D06P 5/00 120 Z;C09D 11/54;C09D 11/32;C09D...,2C056 EA04;2C056 FB03;2C056 FC01;2C056 HA42;2H...,【請求項１】 染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な...,D06P5/00,2018-06-15,0,0,0,1
1,特開2019-218522,インクジェットインク,ゼネラル株式会社,C09D11/30;C09D11/38;B41J2/01;B41M5/00;A61K9/20...,C09D 11/30;C09D 11/38;B41J 2/01 501;B41M ...,2C056 FC01;2H186 BA08;2H186 DA07;2H186 DA12;2H...,【請求項１】 着色剤、アラビアガム、炭素数１２以上、２０以下の高級アルコール、炭素数１以上...,C09D11/30,2018-06-22,1,0,0,0
2,特開2019-218514,インクジェット記録液セット、インクジェット記録用前処理液の製造方法、印刷物及びインクジェット...,コニカミノルタ株式会社,C09D11/54;C09D11/322;B41J2/01;B41M5/00,C09D 11/54;C09D 11/322;B41J 2/01 123;B41J ...,2C056 EA04;2C056 EA13;2C056 FB02;2C056 FC01;2C...,【請求項１】 少なくとも前処理液とインクとからなるインクジェット記録液セットであって、 ...,C09D11/54,2018-06-22,1,0,0,0
3,特開2019-218448,水性インクの製造方法,キヤノン株式会社,C09D11/322;C09D11/326;C09D11/38;B41J2/01;B41M5/00,C09D 11/322;C09D 11/326;C09D 11/38;B41J 2/01 ...,2C056 FC01;2H186 FA20;2H186 FB12;2H186 FB15;2H...,【請求項１】 インクジェット用の水性インクの製造方法であって、 顔料、及び界面活性剤を混...,C09D11/32,2018-06-19,1,0,0,0
4,特開2019-218446,水性インク、インクカートリッジ、記録装置、及び記録方法,富士ゼロックス株式会社,C09D11/322;C09D11/38;B41J2/01;B41M5/00;C09B23/00,C09D 11/322;C09D 11/38;B41J 2/01 501;B41J ...,2C056 FC01;2C056 HA44;2H186 AB12;2H186 BA08;2H...,【請求項１】 水性媒体と、 スクアリリウム骨格を有する化合物と樹脂とを含有する赤外線吸収...,C09D11/32,2018-06-19,1,0,0,0


# 前処理

[正規表現](https://docs.python.org/ja/3/howto/regex.html)を使い、特許文書に特有の文字列を削除します。

処理した文字列をリスト（processed_data）に収納します。

processed_dataをnumpy形式に変換し、保存します。

また、先頭から10文書を表示します。

In [ ]:
import re
import numpy as np
import pandas as pd

#  正規表現のコンパイル
sep = re.compile('【.*?】|\n|\u3000| |\n') 

processed_data = []
for text in data['独立請求項']:
  processed_data.append(sep.sub('', text))
processed_data = np.array(processed_data)
np.save('data/processed_data.npy', processed_data)
processed_data[:10]

array(['染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、色材とを含有する染色調整インクを前記メディアにインクジェット方式により塗布する塗布工程と、前記染色調整インクが塗布された前記メディアを前記染色液に浸漬して染色する染色工程と、を備える、染色物の製造方法。染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、前記染色液による染色工程で視認できなくなるまで除去される色材と、を含有する染色調整インクジェットインク。染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、前記染色液による染色工程で視認可能な程度に残存する色材と、を含有する染色調整インクジェットインク。染色液によるメディアへの着色度合を調整する単独では前記メディア上で視認困難な染色調整剤と、前記染色液による染色工程で視認可能な程度に残存する第１の色材と、を含有する第１の染色調整インクジェットインクと、前記染色調整剤と、前記染...',
       '着色剤、アラビアガム、炭素数１２以上、２０以下の高級アルコール、炭素数１以上、４以下の低級アルコール、および水を含むインクジェットインク。',
       '少なくとも前処理液とインクとからなるインクジェット記録液セットであって、前記前処理液が、少なくとも、ポリエステル、ポリオレフィン又はポリウレタンの骨格のいずれかを含む水不溶性樹脂微粒子と凝集剤と水とを含有し、前記インクが、少なくとも、顔料とオキサゾリン基を有する化合物と水とを含有する、ことを特徴とするインクジェット記録液セット。',
       'インクジェット用の水性インクの製造方法であって、顔料、及び界面活性剤を混合して混合物を得る工程と、得られた前記混合物、及びその他のインク成分を混合する工程と、を有し、前記顔料が、顔料の粒子表面に、（ｉ）アニオン性基、及び、（ｉｉ）他の原子団とアニオン性基とが結合した基、からなる群より選択される官能基が結合した自己分散顔料であるとともに、前記官能基の密度が、０．１０μｍｏｌ／ｍ2以上０．４８μｍｏｌ／ｍ2以下であり、前記界面活性剤が、陽イオンとなりうる窒素原子が第２級アミン又は第３級アミンの構造を有するアミノ酸型界面活性剤であることを特徴と

# 形態素解析

文字列を分解し、スペースで区切った新たな文字列を生成します。

## MeCab

日本語の文法に依存した形態素解析器として最も有名な方法です。

MeCabの導入は環境に依存します。

[mecab-python3](https://pypi.org/project/mecab-python3/) （[Python3からMeCabを使う](https://qiita.com/taroc/items/b9afd914432da08dafc8)）

[natto-py](https://pypi.org/project/natto-py/)（[Python の MeCab バインディング natto-py を使う](https://qiita.com/buruzaemon/items/975027cea6371b2c5ec3)）

等を検索して導入してください。

In [ ]:
# MeCabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

numpy形式のテキストデータを読み込み、

MeCabで分かち書きしたものをリスト（result）に収納します。

In [ ]:
import MeCab
import numpy as pd

m = MeCab.Tagger('-Owakati')

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(m.parse(text))
result = np.array(result)
np.save('data/mecab_data.npy', result)
result[:10]

array(['染色 液 に よる メディア へ の 着色 度合 を 調整 する 単独 で は 前記 メディア 上 で 視認 困難 な 染色 調整 剤 と 、 色材 と を 含有 する 染色 調整 インク を 前記 メディア に インク ジェット 方式 に より 塗布 する 塗布 工程 と 、 前記 染色 調整 インク が 塗布 さ れ た 前記 メディア を 前記 染色 液 に 浸漬 し て 染色 する 染色 工程 と 、 を 備える 、 染色 物 の 製造 方法 。 染色 液 に よる メディア へ の 着色 度合 を 調整 する 単独 で は 前記 メディア 上 で 視認 困難 な 染色 調整 剤 と 、 前記 染色 液 に よる 染色 工程 で 視認 でき なく なる まで 除去 さ れる 色材 と 、 を 含有 する 染色 調整 インク ジェット インク 。 染色 液 に よる メディア へ の 着色 度合 を 調整 する 単独 で は 前記 メディア 上 で 視認 困難 な 染色 調整 剤 と 、 前記 染色 液 に よる 染色 工程 で 視認 可能 な 程度 に 残存 する 色材 と 、 を 含有 する 染色 調整 インク ジェット インク 。 染色 液 に よる メディア へ の 着色 度合 を 調整 する 単独 で は 前記 メディア 上 で 視認 困難 な 染色 調整 剤 と 、 前記 染色 液 に よる 染色 工程 で 視認 可能 な 程度 に 残存 する 第 １ の 色材 と 、 を 含有 する 第 １ の 染色 調整 インク ジェット インク と 、 前記 染色 調整 剤 と 、 前記 染 . . . \n',
       '着色 剤 、 アラビア ガム 、 炭素 数 １２ 以上 、 ２０ 以下 の 高級 アルコール 、 炭素 数 １ 以上 、 ４ 以下 の 低級 アルコール 、 お よび水 を 含む インク ジェット インク 。 \n',
       '少なく とも 前 処理 液 と インク と から なる インク ジェット 記録 液 セット で あっ て 、 前記 前 処理 液 が 、 少なく とも 、 ポリエステル 、 ポリオレフィン 又 は ポリウレタン の 骨格 の いずれ か を 含む 水 不溶 性 樹脂 微 粒子 と 凝集

## [sentencepiece](https://pypi.org/project/sentencepiece/)

サブワード法として最も有名な方法です。

あらかじめ学習済みモデルとして、

[BERT with SentencePiece を日本語 Wikipedia で学習してモデルを公開しました｜原理的には可能](https://yoheikikuta.github.io/bert-japanese/)

が公開している [google Drive](https://drive.google.com/drive/folders/1Zsm9DD40lrUVu6iAnIuTH2ODIkh-WM-O) 上のデータ から 'wiki-ja.model' をダウンロードして　'./data'　に保存しておきます。


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.6MB/s 


In [ ]:
import sentencepiece as spm
import numpy as pd

sp = spm.SentencePieceProcessor()
sp.load('data/wiki-ja.model')

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(' '.join(sp.EncodeAsPieces(text)))
result = np.array(result)
np.save('data/sentencepiece_data.npy', result)
result[:10]

array(['▁ 染色 液 による メディア への 着 色 度 合 を調整する 単独で は 前 記 メディア 上で 視認 困難な 染色 調整 剤 と 、 色 材 と を 含有 する 染色 調整 インク を 前 記 メディア に インク ジェット 方式 により 塗 布 する 塗 布 工程 と 、 前 記 染色 調整 インク が 塗 布 された 前 記 メディア を 前 記 染色 液 に 浸 漬 して 染色 する 染色 工程 と 、 を備える 、 染色 物 の製造 方法 。 染色 液 による メディア への 着 色 度 合 を調整する 単独で は 前 記 メディア 上で 視認 困難な 染色 調整 剤 と 、 前 記 染色 液 による 染色 工程 で 視認 できなくなる まで 除去 される 色 材 と 、 を 含有 する 染色 調整 インク ジェット インク 。 染色 液 による メディア への 着 色 度 合 を調整する 単独で は 前 記 メディア 上で 視認 困難な 染色 調整 剤 と 、 前 記 染色 液 による 染色 工程 で 視認 可能な 程度 に 残存 する 色 材 と 、 を 含有 する 染色 調整 インク ジェット インク 。 染色 液 による メディア への 着 色 度 合 を調整する 単独で は 前 記 メディア 上で 視認 困難な 染色 調整 剤 と 、 前 記 染色 液 による 染色 工程 で 視認 可能な 程度 に 残存 する 第 1 の色 材 と 、 を 含有 する 第 1 の 染色 調整 インク ジェット インク と 、 前 記 染色 調整 剤 と 、 前 記 染 ...',
       '▁ 着 色 剤 、 アラビア ガム 、 炭素 数 12 以上 、 20 以下の 高級 アルコール 、 炭素 数 1 以上 、 4 以下の 低 級 アルコール 、 および 水 を含む インク ジェット インク 。',
       '▁ 少なくとも 前 処理 液 と インク と からなる インク ジェット 記録 液 セット であって 、 前 記 前 処理 液 が 、 少なくとも 、 ポリ エステル 、 ポリ オレ フィン 又は ポリ ウ レ タン の 骨格 のいずれか を含む 水 不 溶 性 樹脂 微 粒子 と 凝 集 剤 と 水 と を 含有 し 、

In [ ]:
# おまけ　テキストを単語ID列に変換することもできます。
import sentencepiece as spm
import numpy as pd

sp = spm.SentencePieceProcessor()
sp.load('data/wiki-ja.model')
result = []
maxlen = 0
for text in processed_data:
  processed_text = sp.EncodeAsIds(text)
  result.append(processed_text)
  if len(processed_text) > maxlen:
    maxlen = len(processed_text)
# zero padding で文書の長さをを揃える
for i, processed_text in enumerate(result):
  result[i] = result[i]+[0]*(maxlen-len(result[i]))
result = np.array(result)
print(result[:10], '\n', result.shape, '次元')

[[    9 25062  3591 ...     0     0     0]
 [    9   447   502 ...     0     0     0]
 [    9  4694   217 ...     0     0     0]
 ...
 [    9  4694 17230 ...     0     0     0]
 [    9 17230  4965 ...     0     0     0]
 [    9   659 12324 ...     0     0     0]] 
 (50, 304) 次元


## n-gram

単純に n文字 で分割しますが、nが2以上の時、重なり合うように切り出す工夫が必要です。

In [ ]:
n = 3 

def ngram(words, n):
  # n文字ずつ切り出し、1文字×nのタプル生成
  # ('染', '色', '液'), ('色', '液', 'に'), ('液', 'に', 'よ')
  ngram = list(zip(*(words[i:] for i in range(n))))
  # n文字の文字列リストに変換　'染色液', '色液に', '液によ', 'による'
  # リストの要素をスペースで連結して一つの文字列にする
  return ' '.join([''.join(j) for j in ngram])

processed_data = np.load('data/processed_data.npy')
result = []
for text in processed_data:
  result.append(ngram(text, n))
result = np.array(result)
np.save('data/'+str(n)+'gram_data.npy', result)
result[:10]


array(['染色液 色液に 液によ による よるメ るメデ メディ ディア ィアへ アへの への着 の着色 着色度 色度合 度合を 合を調 を調整 調整す 整する する単 る単独 単独で 独では では前 は前記 前記メ 記メデ メディ ディア ィア上 ア上で 上で視 で視認 視認困 認困難 困難な 難な染 な染色 染色調 色調整 調整剤 整剤と 剤と、 と、色 、色材 色材と 材とを とを含 を含有 含有す 有する する染 る染色 染色調 色調整 調整イ 整イン インク ンクを クを前 を前記 前記メ 記メデ メディ ディア ィアに アにイ にイン インク ンクジ クジェ ジェッ ェット ット方 ト方式 方式に 式によ により より塗 り塗布 塗布す 布する する塗 る塗布 塗布工 布工程 工程と 程と、 と、前 、前記 前記染 記染色 染色調 色調整 調整イ 整イン インク ンクが クが塗 が塗布 塗布さ 布され された れた前 た前記 前記メ 記メデ メディ ディア ィアを アを前 を前記 前記染 記染色 染色液 色液に 液に浸 に浸漬 浸漬し 漬して して染 て染色 染色す 色する する染 る染色 染色工 色工程 工程と 程と、 と、を 、を備 を備え 備える える、 る、染 、染色 染色物 色物の 物の製 の製造 製造方 造方法 方法。 法。染 。染色 染色液 色液に 液によ による よるメ るメデ メディ ディア ィアへ アへの への着 の着色 着色度 色度合 度合を 合を調 を調整 調整す 整する する単 る単独 単独で 独では では前 は前記 前記メ 記メデ メディ ディア ィア上 ア上で 上で視 で視認 視認困 認困難 困難な 難な染 な染色 染色調 色調整 調整剤 整剤と 剤と、 と、前 、前記 前記染 記染色 染色液 色液に 液によ による よる染 る染色 染色工 色工程 工程で 程で視 で視認 視認で 認でき できな きなく なくな くなる なるま るまで まで除 で除去 除去さ 去され される れる色 る色材 色材と 材と、 と、を 、を含 を含有 含有す 有する する染 る染色 染色調 色調整 調整イ 整イン インク ンクジ クジェ ジェッ ェット ットイ トイン インク ンク。 ク。染 。染色 染色液 色液に 液によ による よるメ 

# ベクトル化

形態素に分割されたテキストを数値化します。

参考：[機械学習 〜 テキスト特徴量（CountVectorizer, TfidfVectorizer） 〜](https://qiita.com/fujin/items/b1a7152c2ec2b4963160)

## Bag of Words

scikit-leran モジュールの [Countvectrizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) を使用して、単語の出現頻度を求めます。

参考：[【python】sklearnのCountVectorizerの使い方｜静かなる名辞](https://www.haya-programming.com/entry/2018/02/25/044525)　ほか多数

注意：

１）すべての文書を読み込んで一気にベクトル化する必要があります。

２）データ量が多いので [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) 形式で出力されます。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word')

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  count_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  savefile = file.replace('_data.npy', '_BoW_csr')
  mmwrite(savefile, csr_matrix(count_vec))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  # 形態素(0-5番目、100-105番目)表示
  print(vectorizer.get_feature_names()[:5], 
        vectorizer.get_feature_names()[100:105], '\n')

MeCabで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (50, 627) 次元 最大値 25
['11', '12', 'μｍ', 'あっ', 'あり'] ['ジェット', 'ジエチル', 'ジメチルアミノプロピルアクリルアミド', 'スクアリリウム', 'スチレン'] 

sentencepieceで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (50, 737) 次元 最大値 24
['00', '10', '100', '11', '12'] ['それぞれ', 'ただし', 'ため', 'ために', 'ための'] 

3-gramで分割した文書をベクトル化
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
 (50, 4907) 次元 最大値 16
['0はｈ', '11の', '11は', '12', '12は'] ['かす走', 'かつ', 'かつｒ', 'から', 'からな'] 



## TF-IDF
scikit-leran モジュールの [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer) を使用して、単語のレア度を求めます。

他の注意事項はCountVectrizer と同様です。

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word')

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  tfidf_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  savefile = file.replace('_data.npy', '_TfIdf_csr')
  mmwrite(savefile, csr_matrix(tfidf_vec))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  # 形態素(0-5番目、100-105番目)表示
  print(vectorizer.get_feature_names()[:5], 
        vectorizer.get_feature_names()[100:105], '\n')

MeCabで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (50, 627) 次元 最大値 0.7843879333604795
['11', '12', 'μｍ', 'あっ', 'あり'] ['ジェット', 'ジエチル', 'ジメチルアミノプロピルアクリルアミド', 'スクアリリウム', 'スチレン'] 

sentencepieceで分割した文書をベクトル化
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.24694266 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] 
 (50, 737) 次元 最大値 0.7466183366424842
['00', '10', '100', '11', '12'] ['それぞれ', 'ただし', 'ため', 'ために', 'ための'] 

3-gramで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0.

CountVectrizer も TfidfVectorizer も入力文の形態素を自動抽出して計算しますが、

vocabulary に単語のリストを与えると、与えた単語に対して計算します。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
vocab = ['備える', 'ノニオン', 'ペロブスカイト', 'ポリエステル', 
         'ポリオレフィン', 'ポリカーボネート', '印刷', '原子', '重なり']

for method, file in files:
  print(method+'で分割した文書をベクトル化', '\nBoW')
  corpus = np.load(file, allow_pickle=True)
  vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', vocabulary=vocab)
  print(vectorizer.get_feature_names())
  count_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  print(count_vec.toarray()[:10], '\n', count_vec.shape, '次元', '最大値', np.max(count_vec.toarray()), '\nTfIdf')
  vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', vocabulary=vocab)
  tfidf_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
  print(tfidf_vec.toarray()[:10], '\n', tfidf_vec.shape, '次元', '最大値', np.max(tfidf_vec.toarray()), '\n')


MeCabで分割した文書をベクトル化 
BoW
['備える', 'ノニオン', 'ペロブスカイト', 'ポリエステル', 'ポリオレフィン', 'ポリカーボネート', '印刷', '原子', '重なり']
[[ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  2  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  0  0  0]] 
 (50, 9) 次元 最大値 11 
TfIdf
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.67902226 0.73411768 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         1.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.   

## HashingVectrizer

[feature hashing](https://ja.wikipedia.org/wiki/Feature_Hashing)という手法を使った埋め込み表現です。

scikit-leran モジュールの [HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) を使用して計算します。

単語情報は設定した(n_features)次元に分散されます。ベクトルの要素と単語とは1対1対応ではありません。

In [2]:
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# MeCab、sentencepiece、3-gram で分割したファイルを読み込みます。
files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]
n_features = [1048576, 1024]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)[:100]
  for i in range(len(n_features)):
    vectorizer = HashingVectorizer(ngram_range=(1, 1), analyzer='word', n_features=n_features[i])
    hash_vec = vectorizer.fit_transform(corpus) # csr_matrix形式
    savefile = file.replace('_data.npy', '_Hash'+str(n_features[i])+'_csr')
    mmwrite(savefile, csr_matrix(hash_vec))
    # 再読込、表示
    loaddata = mmread(savefile+'.mtx').todense()
    print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata))
  print()

MeCabで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (100, 1048576) 次元 最大値 0.7671992600910912
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (100, 1024) 次元 最大値 0.7671992600910912

sentencepieceで分割した文書をベクトル化
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (50, 1048576) 次元 最大値 0.6986670116874985
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.

## Doc2Vec

gensim モジュールの [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html) を用いて、

文章中の隣り合う単語の出現頻度を予測する学習を行い、

得られたモデルに未知の文書を入力すると、学習時に設定した次元のベクトルが得られます。

学習に用いたタグを入力すると、学習に用いた文書のベクトルが得られます。

(参考)

[Doc2Vecについてまとめる](https://qiita.com/g-k/items/5ea94c13281f675302ca)

[Doc2Vecの仕組みとgensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html)

[Word2Vecとは | 分散表現・Skip-gram法とCBOWの仕組み・ツールや活用事例まで徹底解説](https://ledge.ai/word2vec/)

In [ ]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
import joblib

files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
         ]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  training_docs = []
  for i in range(len(corpus)):
    training_docs.append(TaggedDocument(words=corpus[i],
                       tags=[data['公報番号(全文リンク)'][i]]))
  # dm=0 でDoc2Vec 学習
  model = Doc2Vec(documents=training_docs, vector_size=300, min_count=1, dm=0)
  joblib.dump(model, 'data/doc2vec_dm0_'+method+'.model', compress=3)

  # 学習モデルに文書を入力しベクトルを得る
  doc_vec = []
  for i in range(len(data)):
    doc_vec.append(model.infer_vector(training_docs[i].tags))
  savefile = 'data/x_doc2vec_dm0_'+method
  mmwrite(savefile, csr_matrix(np.array(doc_vec)))
  # 再読込、表示
  loaddata = mmread(savefile+'.mtx').todense()
  print(loaddata, '\n', loaddata.shape, '次元', '最大値', np.max(loaddata), '\n')

MeCabで分割した文書をベクトル化
[[-0.00119889 -0.00156107  0.00095843 ...  0.00043311  0.00051889
  -0.00165555]
 [ 0.00083766  0.00089225 -0.00108992 ... -0.00050002 -0.00135353
   0.00035542]
 [ 0.00138262 -0.00062317  0.00074927 ...  0.00062413 -0.00128545
  -0.00150257]
 ...
 [ 0.0012527   0.00055048 -0.00115774 ... -0.00150503  0.00158021
  -0.00068571]
 [-0.00044679  0.00072698 -0.00121772 ... -0.0009695  -0.00015943
   0.00031917]
 [ 0.00143875 -0.00037791 -0.00137032 ...  0.00124565 -0.00107917
  -0.00156544]] 
 (50, 300) 次元 最大値 0.0016665872 

sentencepieceで分割した文書をベクトル化
[[-0.00119889 -0.00156107  0.00095843 ...  0.00043311  0.00051889
  -0.00165555]
 [ 0.00083766  0.00089225 -0.00108992 ... -0.00050002 -0.00135353
   0.00035542]
 [ 0.00138262 -0.00062317  0.00074927 ...  0.00062413 -0.00128545
  -0.00150257]
 ...
 [ 0.0012527   0.00055048 -0.00115774 ... -0.00150503  0.00158021
  -0.00068571]
 [-0.00044679  0.00072698 -0.00121772 ... -0.0009695  -0.00015943
   0.00031917]
 [ 0.00143875 -0.0

## ELMo

双方向LSTMを用いて学習させた言語モデルで、文脈を考慮した単語埋め込み表現が得られます。[実装](https://pypi.org/project/elmoformanylangs/)

（参考）

[大規模日本語ビジネスニュースコーパスを学習したELMo（MeCab利用）モデルの紹介](https://qiita.com/mkt3/items/9577b63900109ff91665)

[大規模日本語ビジネスニュースコーパスを学習したELMo（MeCab利用）モデルの利用方法と精度比較検証](https://qiita.com/kaeru_nantoka/items/bca53a2daea2b29c9b39)

In [2]:
# 必要なライブラリをインストールします。
%cd '/content/drive/My Drive/'
!pip install overrides
!git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git
!sudo python 'ELMoForManyLangs/setup.py' install


# 必要なライブラリをインポートします。
import numpy as np
import pandas as pd
import torch
from ELMoForManyLangs.elmoformanylangs import Embedder
from overrides import overrides
from IPython.display import clear_output

/content
fatal: destination path 'ELMoForManyLangs' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing elmoformanylangs.egg-info/PKG-INFO
writing dependency_links to elmoformanylangs.egg-info/dependency_links.txt
writing requirements to elmoformanylangs.egg-info/requires.txt
writing top-level names to elmoformanylangs.egg-info/top_level.txt
reading manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying elmoformanylangs.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying elmoformanylangs.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying elmoformanylangs.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying elmoformanylangs.egg-info/requires.txt -> 

（作業）

https://drive.google.com/drive/u/1/folders/1sau1I10rFeAn8BDk8eZDL5qaEjTlNghp

こちらから　単語単位埋め込みモデル　と　文字単位・単語単位埋め込みモデル　とをダウンロードし、マイドライブにアップロードします。

In [55]:
# ELMo

def serial_mode(txt_batch):
  data = []
  for k in range(len(txt_batch)):
    torch.cuda.empty_cache()
    data.append(char_e.sents2elmo([txt_batch[k]], output_layer=output_layer))
  return data

word_model_path = './単語単位埋め込みモデル'
char_model_path = './文字単位・単語単位埋め込みモデル'

# 文字単位・単語単位埋め込みモデルを読み込み
char_e = Embedder(model_dir=char_model_path, batch_size=64)
output_layer=-1 # モデルの最終段の出力を得る

corpus = np.load('data/mecab_data.npy', allow_pickle=True)

texts = []
new_array = []
n_batch = 50
print('\nコーパス整理中')
for i in range(len(corpus)):
  texts.append(corpus[i].split(' '))

for i in range(len(corpus)//n_batch):
  if i%30 == 0:
    clear_output()
  print('\rELMo変換中\t', 
        str(n_batch*i)+'-'+str(n_batch*(i+1))+'/'+str(len(corpus)), end='\t')
  try:
    data = char_e.sents2elmo(texts[n_batch*i:n_batch*(i+1)], output_layer=output_layer)
    # 可変長ベクトルの平均をとり固定長ベクトルにする
    for j in range(len(data)):
      new_array.append(np.average(data[j], axis=0).reshape(1,-1))
    torch.cuda.empty_cache()
  except:
    # n_batch数の処理でメモリオーバーフローを起こすとき1文書ずつ処理する
    print('serial mode')
    data = serial_mode(texts[n_batch*i:n_batch*i+1])
    for j in range(len(data)):
      new_array.append(np.average(data[j][0], axis=0).reshape(1,-1))
      torch.cuda.empty_cache()
    print('\rELMo変換中\t', 
        str(n_batch*i)+'-'+str(len(corpus))+'/'+str(len(corpus)), end='\t')
data = serial_mode(texts[n_batch*(i+1):])
for j in range(len(data)):
  new_array.append(np.average(data[j][0], axis=0).reshape(1,-1))

new_array = np.vstack(new_array)
np.save('data/x_ELMo_mecab.npy', new_array)
print('実行結果', new_array.shape, '\n', new_array)

ELMo変換中	 4500-4550/5351	

2020-10-08 22:30:12,219 INFO: 1 batches, avg len: 192.0


ELMo変換中	 4550-4600/5351	

2020-10-08 22:30:15,052 INFO: 1 batches, avg len: 186.6


ELMo変換中	 4600-4650/5351	

...

serial mode
ELMo変換中	 5300-5350/5351	

2020-10-08 22:30:54,603 INFO: 1 batches, avg len: 377.2
2020-10-08 22:30:54,633 INFO: 1 batches, avg len: 86.0


serial mode


2020-10-08 22:30:55,349 INFO: 1 batches, avg len: 467.0


ELMo変換中	 5300-5351/5351	実行結果 (5204, 1024) 
 [[-0.33146614  0.20417939  0.20073496 ...  0.6192389   0.22071181
  -0.10123414]
 [-0.02435132  0.28076112  0.06769402 ...  0.7251096   0.15409651
  -0.25710547]
 [-0.01607116  0.21535724  0.41298237 ...  0.67664057  0.4805104
   0.18667226]
 ...
 [ 0.06666844  0.4897722   0.19548763 ...  0.7980848   0.2269311
  -0.18554683]
 [-0.06452872  0.41726747  0.14705794 ...  0.72102714  0.19435036
  -0.04705095]
 [-0.15871945  0.1290972   0.35129866 ...  0.60404783  0.05334947
  -0.43363634]]


In [53]:
np.average(data[j][0], axis=0).shape

(1024,)

# sentence-transformers日本語版

https://github.com/sonoisa/sentence-transformers

（参考）

[はじめての自然言語処理｜第9回 Sentence BERT による類似文章検索の検証](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part9.html)

In [2]:
! pip install mecab-python3==0.996.5

     |████████████████████████████████| 17.1MB 229kB/s 


In [3]:
%cd '/content/drive/My Drive/'
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt

/content
fatal: destination path 'sentence-transformers' already exists and is not an empty directory.
     |████████████████████████████████| 450kB 4.6MB/s 
     |████████████████████████████████| 890kB 19.0MB/s 
     |████████████████████████████████| 1.1MB 29.6MB/s 
     |████████████████████████████████| 133kB 40.7MB/s 
     |████████████████████████████████| 6.7MB 33.9MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b3cb24ca4d12bd52d0b7039c6941e9dce0e674b08350d50ba242e2826783ff4d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar
%cd '/content/drive/My Drive/sentence-transformers/'
from sentence_transformers import SentenceTransformer
%cd '/content/drive/My Drive/'


--2020-10-08 22:42:57--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2
Resolving www.floydhub.com (www.floydhub.com)... 104.26.0.30, 172.67.72.144, 104.26.1.30, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.26.0.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘sonobe-datasets-sentence-transformers-model.tar’

sonobe-datasets-sen     [                <=> ] 422.28M  40.6MB/s    in 10s     

2020-10-08 22:43:07 (42.2 MB/s) - ‘sonobe-datasets-sentence-transformers-model.tar’ saved [442788352]

./
./training_bert_japanese/
./training_bert_japanese/0_BERTJapanese/
./training_bert_japanese/0_BERTJapanese/added_tokens.json
./training_bert_japanese/0_BERTJapanese/config.json
./training_bert_japanese/0_BERTJapanese/pytorch_model.bin
./training_bert_japanese/0_BERTJapanese/sentence_bert_config.json
./training_bert_japa

In [7]:
%tensorflow_version 2.x
import numpy as np
model_path = '/content/drive/My Drive/training_bert_japanese'
model = SentenceTransformer(model_path, show_progress_bar=False)


In [14]:
# document vector
import joblib
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

corpus = np.load('data/mecab_data.npy', allow_pickle=True)

vectors = model.encode(corpus)
vectors = np.vstack(vectors)
np.save('data/x_ELMo_mecab.npy', vectors)
print('実行結果', vectors.shape, '\n', vectors)


実行結果 (5328, 768) 
 [[ 0.30623767  0.9199255  -0.3692814  ... -0.45889604 -0.18640393
  -1.1709334 ]
 [ 0.74890316 -0.36533457 -0.14062168 ... -0.44990322 -0.39594465
  -0.5531663 ]
 [ 0.57097554 -0.07667922 -0.4922952  ... -0.4587596  -0.92465496
  -1.009658  ]
 ...
 [ 0.04443025  0.57329595 -0.780079   ...  0.75817376 -0.42281306
  -1.6895096 ]
 [ 0.95789146 -0.29498622 -1.1318222  ...  0.7242595   0.4687451
  -1.7129064 ]
 [ 0.6381206   0.0237502  -0.5740135  ...  0.4319682  -0.42214027
  -0.73637563]]


## ラベルエンコード、one-hot ベクトル

Tensorflow.Kerasモジュールのテキスト処理クラスである[Tokenizer](tf.keras.preprocessing.text.Tokenizer)に文書を与えることによって、

単語IDの列からなる文書ベクトルが得られます。

このとき、sequenceで各文書の長さをそろえますが、短い文書の後ろに 0 を付ける（zero padding）のが主流です。

長い文書に揃えず、適当な長さで区切って、余った単語を切り捨てる方法もあります。

参考：[Keras Documentation](https://keras.io/ja/preprocessing/text/)

また、[np_utilsクラス](https://keras.io/ja/utils/np_utils/)の to_categorical API に数値nを入力すると、

n番目が1で残りが0のベクトル（one-hotベクトル）が得られます。

参考：[Keras Documentation](https://keras.io/ja/)

In [ ]:
# ラベルエンコード

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

files = [['MeCab', 'data/mecab_data.npy'], 
         ['sentencepiece', 'data/sentencepiece_data.npy'], 
         ['3-gram', 'data/3gram_data.npy']
]

for method, file in files:
  print(method+'で分割した文書をベクトル化')
  corpus = np.load(file, allow_pickle=True)
  # 文字列をID列に変換
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  list_tokenized = tokenizer.texts_to_sequences(corpus)
  # ID列の長さを揃える
  list_sequence = sequence.pad_sequences(list_tokenized, maxlen=None, dtype='int32', 
                                        padding='post', value=0.0)
  list_tokenized = np.array(list_sequence)
  print(list_tokenized, '\n', list_tokenized.shape, '次元', '最大値', np.max(list_tokenized), '\n')
  # ID列の長さを50で打ち切る
  list_sequence = sequence.pad_sequences(list_tokenized, maxlen=50, dtype='int32', 
                                        padding='post', value=0.0)
  list_tokenized = np.array(list_sequence)
  print(list_tokenized, '\n', list_tokenized.shape, '次元', '最大値', np.max(list_tokenized), '\n')  

MeCabで分割した文書をベクトル化
[[ 59  92   5 ...   0   0   0]
 [155  55   1 ...   0   0   0]
 [ 43  40 202 ...   0   0   0]
 ...
 [  6 133   3 ...   0   0   0]
 [103 132 107 ...   0   0   0]
 [342 343  48 ...   0   0   0]] 
 (50, 307) 次元 最大値 796 

[[ 55   7   1 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [133   3 168 ...   0   0   0]
 [  4 198 559 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 
 (50, 50) 次元 最大値 791 

sentencepieceで分割した文書をベクトル化
[[ 27  49  42 ...   0   0   0]
 [ 27  63  89 ...   0   0   0]
 [ 27  37   3 ...   0   0   0]
 ...
 [ 27   4  42 ...   0   0   0]
 [ 27  92 137 ...   0   0   0]
 [ 27 383 384 ...   0   0   0]] 
 (50, 281) 次元 最大値 1000 

[[  8  49 133 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [ 34  10   4 ...   0   0   0]
 [  1 128   9 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 
 (50, 50) 次元 最大値 992 

3-gramで分割した文書をベクトル化
[[ 256  257  304 ...    0    0    0]
 [2719 2720 2721 ...    0    0

In [ ]:
# one-hotベクトル(1)
# Tensorflowに取り込まれる前のKerasのutilityにあるnp_utilsを用いる方法

from keras.utils import np_utils

index_data = [0, 1, 0, 2, 0, 1, 0, 2]
np_utils.to_categorical(index_data)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
# one-hotベクトル(2)
# 単位行列を生成するNumPyのeye関数で単位行列を生成し、数値をかける方法

import numpy as np

index_data = [0, 1, 0, 2, 0, 1, 0, 2]
print(np.eye(max(index_data)+1)) # 3×3単位行列
np.eye(max(index_data)+1)[index_data]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])